In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dependencies 
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

Read in CSV File

In [3]:
# Source: https://www.kaggle.com/datasets/russellyates88/suicide-rates-overview-1985-to-2016?resource=download
columns = [
    "country","year","sex","age","suicides_no","population","suicides/100k_pop","country_year",
    "HDI_four_year","gdp_per_capita ($)","generation"
]

target = ["year"]

In [4]:
# Load the data
file_path = ('master.csv')
df = pd.read_csv(file_path)

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df.reset_index(inplace=True, drop=True)

df

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1995,male,25-34 years,13,232900,5.58,Albania1995,0.619,"2,424,499,009",835,Generation X
1,Albania,1995,male,55-74 years,9,178000,5.06,Albania1995,0.619,"2,424,499,009",835,Silent
2,Albania,1995,female,75+ years,2,40800,4.90,Albania1995,0.619,"2,424,499,009",835,G.I. Generation
3,Albania,1995,female,15-24 years,13,283500,4.59,Albania1995,0.619,"2,424,499,009",835,Generation X
4,Albania,1995,male,15-24 years,11,241200,4.56,Albania1995,0.619,"2,424,499,009",835,Generation X
...,...,...,...,...,...,...,...,...,...,...,...,...
8359,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation X
8360,Uzbekistan,2014,female,75+ years,9,348465,2.58,Uzbekistan2014,0.675,"63,067,077,179",2309,Silent
8361,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z
8362,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z


Split the Data into Training and Testing

In [5]:
print(df.columns.tolist())

['country', 'year', 'sex', 'age', 'suicides_no', 'population', 'suicides/100k pop', 'country-year', 'HDI for year', ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation']


In [7]:
# Create our features
X = df[["suicides_no"]]

# Create our target
y = df["age"]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(6273, 1)

Using Easy Ensemble AdaBoost Classifier

In [9]:
# Resample the training data with the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
suicides_model = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
suicides_model

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [10]:
# Fit the model
suicides_model = suicides_model.fit(X_train, y_train)
y_pred = suicides_model.predict(X_test)

In [11]:
# Check balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.27052388323507776

In [12]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[138,   0,  66,  80,  65,   0],
       [144,   0,  72,  75,  57,   0],
       [113,   0,  63,  63, 109,   0],
       [ 80,   0,   6, 262,   1,   0],
       [101,   0,  58,  86, 103,   0],
       [117,   0,  50, 132,  50,   0]])

In [13]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

15-24 years       0.20      0.40      0.68      0.26      0.52      0.26       349
25-34 years       0.00      0.00      1.00      0.00      0.00      0.00       348
35-54 years       0.20      0.18      0.86      0.19      0.39      0.14       348
 5-14 years       0.38      0.75      0.75      0.50      0.75      0.56       349
55-74 years       0.27      0.30      0.84      0.28      0.50      0.23       348
  75+ years       0.00      0.00      1.00      0.00      0.00      0.00       349

avg / total       0.17      0.27      0.85      0.21      0.36      0.20      2091



Using BalancedRandomForestClassifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
suicides_model = BalancedRandomForestClassifier(n_estimators = 100, random_state=1)
suicides_model

BalancedRandomForestClassifier(random_state=1)

In [15]:
# Fit the model
suicides_model = suicides_model.fit(X_train, y_train)

In [16]:
# Calculated the balanced accuracy score
y_pred = suicides_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.2767828716969118

In [17]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 64,  82,  36,  92,  42,  33],
       [ 60,  79,  43,  84,  51,  31],
       [ 47,  86,  65,  70,  40,  40],
       [ 21,  31,   7, 270,   9,  11],
       [ 54,  54,  53,  95,  59,  33],
       [ 43,  59,  42, 138,  25,  42]])

In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

15-24 years       0.22      0.18      0.87      0.20      0.40      0.15       349
25-34 years       0.20      0.23      0.82      0.21      0.43      0.18       348
35-54 years       0.26      0.19      0.90      0.22      0.41      0.16       348
 5-14 years       0.36      0.77      0.73      0.49      0.75      0.56       349
55-74 years       0.26      0.17      0.90      0.21      0.39      0.14       348
  75+ years       0.22      0.12      0.92      0.16      0.33      0.10       349

avg / total       0.26      0.28      0.86      0.25      0.45      0.21      2091

